# Create databases

This will create databases from root schema for testing clearscape analytics

Database hierarchy is
CS01
CS01_RAW
CS01_RAW_VIEW
CS01_TARGET
CS01_TARGET_VIEW

In [17]:
import teradatasql
import yaml
import sys
import os
import sqlparse
import pandas as pd



In [18]:
# get home directory
from os.path import expanduser
import pprint
home = expanduser("~")
# assume project is called dbt_clearscape
project = 'dbt_clearscape'
# read config file
with open(home + '/.dbt/profiles.yml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
# get project config
if project not in config:
    print('project not found in config file')
project_config = config[project]
# get target
target = project_config['target']
# get connection details
project_connection = project_config['outputs'][target]
#pprint.pprint(project_connection)


Connect to database

In [20]:
# connect to teradata
conn = teradatasql.connect(host=project_connection['server'], user=project_connection['username'], password=project_connection['password'])    

OperationalError: [Version 20.0.0.19] [Session 0] [Teradata SQL Driver] Hostname lookup failed for primarkclw-vjpiuc437tcyfda4.env.clearscape.teradata.com
 at gosqldriver/teradatasql.formatError ErrorUtil.go:91
 at gosqldriver/teradatasql.(*teradataConnection).makeDriverError ErrorUtil.go:193
 at gosqldriver/teradatasql.(*nameLookup).getAddresses CopDiscovery.go:95
 at gosqldriver/teradatasql.discoverCops CopDiscovery.go:177
 at gosqldriver/teradatasql.newTeradataConnection TeradataConnection.go:145
 at gosqldriver/teradatasql.(*teradataDriver).Open TeradataDriver.go:32
 at database/sql.dsnConnector.Connect sql.go:791
 at database/sql.(*DB).conn sql.go:1415
 at database/sql.(*DB).Conn.func1 sql.go:1940
 at database/sql.(*DB).retry sql.go:1566
 at database/sql.(*DB).Conn sql.go:1939
 at main.goCreateConnection goside.go:303
 at _cgoexp_e3ee842aae7c_goCreateConnection _cgo_gotypes.go:265
 at runtime.cgocallbackg1 cgocall.go:403
 at runtime.cgocallbackg cgocall.go:322
 at runtime.cgocallb

# Create all the database

The databases are

- CS01
- CS01_RAW
- CS01_RAW_VIEW
- CS01_TARGET
- CS01_TARGET_VIEW



In [11]:
# TODO - setup parameterised db config
db_list = {}
db_list ['CS01'] = {'owner': {project_connection['username']},'space':0}
db_list ['CS01_RAW'] = {'owner': 'CS01','space':0}
db_list ['CS01_RAW_VIEW'] = {'owner': 'CS01','space':0}
db_list ['CS01_TARGET'] = {'owner': 'CS01','space':10000}
db_list ['CS01_TARGET_VIEW'] = {'owner': 'CS01','space':0}

In [13]:
# Delete and then drop all databases
with conn.cursor() as csr:
    for i,db in enumerate(db_list):
        print(i,db)
        csr.execute(f'DELETE DATABASE {db}')
        csr.execute(f'DROP DATABASE {db}')

0 CS01


OperationalError: [Version 20.0.0.19] [Session 1624] [Teradata SQL Driver] Failure sending Start Request message
 at gosqldriver/teradatasql.formatError ErrorUtil.go:91
 at gosqldriver/teradatasql.(*teradataConnection).makeDriverError ErrorUtil.go:193
 at gosqldriver/teradatasql.(*teradataConnection).socketSendMessage NetworkIO.go:291
 at gosqldriver/teradatasql.(*teradataConnection).socketSendAndReceiveContext NetworkIO.go:70
 at gosqldriver/teradatasql.(*teradataConnection).sendAndReceiveContext NetworkIO.go:50
 at gosqldriver/teradatasql.(*TeradataRows).executeSQLRequest TeradataRows.go:846
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:720
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryContext TeradataConnection.go:1225
 at database/sql.ctxDriverQuery ctxutil.go:48
 at database/sql.(*DB).queryDC.func1 sql.go:1776
 at database/sql.withLock sql.go:3530
 at database/sql.(*DB).qu

In [ ]:
# TODO : Convert to a more parameteris version

In [25]:
sql = f"CREATE DATABASE CS01 FROM {project_connection['username']} AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01 AS 'Base database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [26]:
sql = f"CREATE DATABASE CS01_RAW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_RAW AS 'raw database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [27]:
sql = f"CREATE DATABASE CS01_RAW_VIEW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_RAW_VIEW AS 'raw database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [28]:
sql = f"CREATE DATABASE CS01_TARGET FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_TARGET AS 'target database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [29]:
sql = f"CREATE DATABASE CS01_TARGET_VIEW FROM CS01 AS PERM=0"
with conn.cursor() as cur:
    cur.execute(sql, ignoreErrors=[5612])
    print('Database created')
    cur.execute("COMMENT ON CS01_TARGET_VIEW AS 'target database for Clearscape'")
    print('Comment added')

Database created
Comment added


In [30]:
# grant intra database access
sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_RAW TO CS01_RAW_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

sql = f"GRANT SELECT, INSERT, UPDATE, DELETE ON CS01_TARGET TO CS01_TARGET_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')

# dbc access
sql = f"GRANT SELECT ON DBC.Databases TO CS01_RAW_VIEW WITH GRANT OPTION"
with conn.cursor() as cur:
    cur.execute(sql)
    print('Grant created')



Grant created
Grant created


Allocate space for the databases

In [9]:
with conn.cursor() as csr:
    # drop space database if it exists
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [3802])
    # create space data with space
    csr.execute(f'CREATE DATABASE CS01_SPACE FROM {project_connection['username']}  AS PERM=10000000 ')
    # give space to database
    csr.execute(f'GIVE CS01_SPACE TO CS01_RAW')
    # drop space
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [])

In [10]:
with conn.cursor() as csr:
    # drop space database if it exists
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [3802])
    # create space data with space
    csr.execute(f'CREATE DATABASE CS01_SPACE FROM {project_connection['username']}  AS PERM=10000000 ')
    # give space to database
    csr.execute(f'GIVE CS01_SPACE TO CS01_TARGET')
    # drop space
    csr.execute('DROP DATABASE CS01_SPACE',ignoreErrors = [])